**Author :** Rutendo F. Sigauke

**Input  :** 
1. Count files for *each* sample in DBNascent (Genes and Bidirectionals)

        a. Genes stranded 5' truncated (filt.5ptrunc_gene.stranded_counts.txt)
        b. Genes stranded full length (filt.gene.stranded_counts.txt)
        c. Bidirectional transcripts (bidir.pos_counts.txt' and 'bidir.neg_counts.txt)
    
2. Annotation files (Genes and Bidirectionals)

**Output :**

1. Merged count files 

        a. Genes stranded 5' truncated (counts_filt_5ptrunc_gene_stranded_counts.txt)
        b. Genes stranded full length (counts_filt_gene_stranded_counts.txt)
        c. Bidirectional transcripts (counts_bidirectionals_pos.bed and counts_bidirectionals_neg.bed)

# Introduction

Counts from nascent RNA sequencing datasets in `DBNascent` are merged to a single text file.

The count data is in `/Shares/dbnascent/{paperYearID}/` in `featurecounts_bidirs` and `featurecounts_genes`. In each folder, for each sample counts tables are present.

There are two gene count types **all** where all annotated gene transcripts are used and **filt** where manually curated gene transripts are used. The **5ptrunc** are transcript annotations where the 5' end of the gene was removed. Lastly, counts were either **stranded** or **unstranded**.
 
e.g.:
```sh

rusi2317@fiji-2.colorado.edu$ pwd
/Shares/dbnascent/Andrysik2017identification/featurecounts_genes

rusi2317@fiji-2.colorado.edu$ ls
SRR4090098.all.5ptrunc_gene.stranded_counts.txt
SRR4090098.all.5ptrunc_gene.unstranded_counts.txt
SRR4090098.all.gene.stranded_counts.txt
SRR4090098.all.gene.unstranded_counts.txt
SRR4090098.filt.5ptrunc_gene.stranded_counts.txt
SRR4090098.filt.5ptrunc_gene.unstranded_counts.txt
SRR4090098.filt.gene.stranded_counts.txt
SRR4090098.filt.gene.unstranded_counts.txt

```

Bidirectional counts are split into stranded pos/neg and unstranded counts.

```sh
rusi2317@fiji-2.colorado.edu$ pwd
/Shares/dbnascent/Andrysik2017identification/featurecounts_bidirs

rusi2317@fiji-2.colorado.edu$ ls
SRR4090098.bidir.neg_counts.txt
SRR4090098.bidir.pos_counts.txt
SRR4090098.bidir.unstranded_counts.txt

```

# Import libraries

In [1]:
library(dplyr) ##data.frame wrangling
library(tidyr) ##data.frame wrangling
library(data.table) ##load large data.frames


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




# Annotations

## RefSeq genes

### 5' truncated

In [2]:
hg38_refseq_bed <- data.table::fread('/scratch/Shares/dowell/genomes/hg38/ncbi/hg38_refseq_diff53prime_with_putatives_5ptrunc.bed',
                              sep='\t')
colnames(hg38_refseq_bed) <- c("chrom","start","stop", 
                               "gene_transcript","score","strand")

##since the same transcript start stops are separated by comma, we can select the first transcript ID as shown below
hg38_refseq_bed$gene_transcript <- as.character(lapply(strsplit(hg38_refseq_bed$gene_transcript, ','), `[`, 1))

##adding lengths and gene ids
hg38_refseq_bed$length <- hg38_refseq_bed$stop - hg38_refseq_bed$start
hg38_refseq_bed$gene <- as.character(lapply(strsplit(hg38_refseq_bed$gene_transcript, ':'), `[`, 1))
dim(hg38_refseq_bed)
head(hg38_refseq_bed, 3)

[1] 83826     8

chrom,start,stop,gene_transcript,score,strand,length,gene
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
chr1,12623,14409,DDX11L1:NR_046018.2,.,+,1786,DDX11L1
chr1,14361,28620,WASH7P:NR_024540.1,.,-,14259,WASH7P
chr1,17368,17436,MIR6859-1:NR_106918.1,.,-,68,MIR6859-1


### Full length

In [3]:
hg38_refseq_full_bed <- data.table::fread('/scratch/Shares/dowell/genomes/hg38/ncbi/hg38_refseq_diff53prime_with_putatives.bed',
                              sep='\t')
colnames(hg38_refseq_full_bed) <- c("chrom","start","stop", 
                               "gene_transcript","score","strand")

##since the same transcript start stops are separated by comma, we can select the first transcript ID as shown below
hg38_refseq_full_bed$gene_transcript <- as.character(lapply(strsplit(hg38_refseq_full_bed$gene_transcript, ','), `[`, 1))

##adding lengths and gene ids
hg38_refseq_full_bed$length <- hg38_refseq_full_bed$stop - hg38_refseq_full_bed$start
hg38_refseq_full_bed$gene <- as.character(lapply(strsplit(hg38_refseq_full_bed$gene_transcript, ':'), `[`, 1))
dim(hg38_refseq_full_bed)
head(hg38_refseq_full_bed, 3)

[1] 83826     8

chrom,start,stop,gene_transcript,score,strand,length,gene
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
chr1,11873,14409,DDX11L1:NR_046018.2,.,+,2536,DDX11L1
chr1,14361,29370,WASH7P:NR_024540.1,.,-,15009,WASH7P
chr1,17368,17436,MIR6859-1:NR_106918.1,.,-,68,MIR6859-1


## Bidirectional transcripts

In [4]:
##example bed 6 file from one of the bed6 files
hg38_bidirectionals <- data.table::fread('/scratch/Shares/dowell/dbnascent/out/meta_analysis/mumerge/bidirectionals_dreg_tfit/hg38_tfit_dreg_bidirectionals.bed')
colnames(hg38_bidirectionals) <- c('chrom','start','end','bidirs','score','strand')

dim(hg38_bidirectionals)
head(hg38_bidirectionals, 3)

[1] 847521      6

chrom,start,end,bidirs,score,strand
<chr>,<int>,<int>,<chr>,<int>,<chr>
chr1,3917,4919,dreg,14,.
chr1,5632,6042,dreg,14,.
chr1,6132,6486,dreg,7,.


# Functions 

## Load all the files to one

In [5]:
merge_files <- function(file_directory, file_pattern, drop_list, ntranscripts, drop=FALSE){
    
    #assign path for directory to variable
    file_dir <- file_directory
    
    #list all files in that directory based on a pattern
    file_paths <- list.files(path=file_dir,
                             pattern=file_pattern,
                             recursive = TRUE,
                             full.names=TRUE)
    file_names <- base::basename(file_paths)
    
    print(paste0("Input files : ", as.character(length(file_paths))))
    
    #load all files excluding the metadata columns (if you wish)
    if (drop == TRUE) {
        file_DT_list <- lapply(file_paths,
                               data.table::fread,
                               drop = drop_list)
        } else {
        file_DT_list <- lapply(file_paths,
                               data.table::fread)
        
    }
    
    ##make sure the counts are from human annotations with ( human == 42224 transcripts)
    file_DT_list_checked <- file_DT_list[sapply(file_DT_list,
                                          function(x) nrow(x) == ntranscripts)]
                                                
    ##merge all counts for human samples
    file_DT_features_one <- dplyr::bind_cols(file_DT_list_checked)
    file_DT_newnames <- setnames(file_DT_features_one, names(file_DT_features_one),
                               as.character(lapply(strsplit(names(file_DT_features_one), '\\_'), `[`, 1)))
                                                
    print(paste0("Loaded files matching # transcripts : ",
                 as.character(ncol(file_DT_newnames))))
                                                
    same_file <- ncol(file_DT_newnames)==length(file_paths)
                                                
    print(paste("Did the same number of files get loaded? : ",
                same_file))

    if (same_file == FALSE){
        print("The difference is due to human vs. mouse samples in each folder")
    }
    
    return(file_DT_newnames)
}

## Combine counts with RefSeq annotations

In [6]:
example_sample <- data.table::fread('/Shares/dbnascent/Andrysik2017identification/featurecounts_genes/SRR4090098.filt.5ptrunc_gene.stranded_counts.txt')
example_sample$gene_transcript <- paste0(example_sample$gene_id,":",example_sample$transcript_id)
dim(example_sample)
head(example_sample, 3)

[1] 42224     5

gene_id,transcript_id,length,SRR4090098_counts,gene_transcript
<chr>,<chr>,<int>,<int>,<chr>
DDX11L1,NR_046018.2,1786,0,DDX11L1:NR_046018.2
WASH7P,NR_024540.1,14259,40,WASH7P:NR_024540.1
MIR6859-1,NR_106918.1,68,0,MIR6859-1:NR_106918.1


In [7]:
example_sample_full <- data.table::fread('/Shares/dbnascent/Andrysik2017identification/featurecounts_genes/SRR4090098.filt.gene.stranded_counts.txt')
example_sample_full$gene_transcript <- paste0(example_sample_full$gene_id,":",example_sample_full$transcript_id)
dim(example_sample_full)
head(example_sample_full, 3)

[1] 42224     5

gene_id,transcript_id,length,SRR4090098_counts,gene_transcript
<chr>,<chr>,<int>,<int>,<chr>
DDX11L1,NR_046018.2,2536,0,DDX11L1:NR_046018.2
WASH7P,NR_024540.1,15009,40,WASH7P:NR_024540.1
MIR6859-1,NR_106918.1,68,0,MIR6859-1:NR_106918.1


In [8]:
combine_anno_counts <- function(file_directory, file_pattern, 
                                out_directory, out_file, 
                                ntranscripts, transcript_beds_DT, sample){
    
    wd <- out_directory
    dropped <- c('gene_id', 'transcript_id', 'length')
    
    counts_DT <- merge_files(file_directory=file_directory, 
                             file_pattern=file_pattern, 
                             drop_list=dropped,
                             ntranscripts=ntranscripts,
                             drop=TRUE)
    
    print(paste("Number of samples = ", ncol(counts_DT))) 
    
    ##merge the counts and refseq annotations by gene:transcipt IDs
    counts_DT$gene_transcript <- sample$gene_transcript

    counts_DT_features <- merge(transcript_beds_DT,  
                                counts_DT, 
                                by="gene_transcript")

    ##Select counts from the longest gene transcript
    counts_DT_genes <- counts_DT_features[order(-gene, length), 
                                      head(.SD, 1), by = gene]
    
    order_DT_all <- c('chrom','start','stop','gene_transcript','score',
                      'strand', 
                      names(counts_DT))
    
    order_DT <- head(order_DT_all, -1)
    
    print(paste("Number of columns = ", length(order_DT))) 
    
    print("--------------------------")
    print("> Columns AFTER filtering")
    print(order_DT)
    print("--------------------------")
    
    ##subset the counts file in a bed 6 file format
    ##plus the counts in remaining columns
    counts_DT_subset <- counts_DT_genes[ ,colnames(counts_DT_genes) %in% 
                                        order_DT, with=FALSE]
    
    ##rearrange bed file to match the bed 6 + samples format
    counts_DT_bed <- data.table::setcolorder(counts_DT_subset, order_DT)
    
    ##order the transcripts by genomics coordinates chromsome and start
    counts_DT_bed_order <- counts_DT_bed[order(rank(chrom), start)]
    
    ##saving the merged file
    data.table::fwrite(counts_DT_bed_order, sep='\t',
                      file = paste0(wd, out_file))
    
}

## Combine bidirectionals counts

In [9]:
combine_bidir_counts <- function(file_directory, file_pattern, 
                                out_directory, out_file, 
                                ntranscripts, transcript_beds_DT){
    
    print("--------------------------")
    wd <- out_directory
    dropped <- c('bidir_id')
    
    ##merge all counts to single DT
    counts_DT <- merge_files(file_directory=file_directory, 
                             file_pattern=file_pattern, 
                             drop_list=dropped,
                             ntranscripts=ntranscripts,
                             drop=TRUE)
    
    ##count number of samples with counts
    print(paste("Number of samples = ", ncol(counts_DT))) 
    
    ##combine the annotations with counts
    counts_DT_features <- cbind(transcript_beds_DT,  
                                counts_DT)
    
    ##order the transcripts by genomics coordinates chromsome and start
    counts_DT_bed_order <- counts_DT_features[order(rank(chrom), start)]
    
    ##saving the merged file
    data.table::fwrite(counts_DT_bed_order, sep='\t',
                      file = paste0(wd, out_file))
    print("--------------------------")
}

# Merge counts

## Genes

### 5' Truncated

In [10]:
shares <- '/Shares/dbnascent'
wd <- '/Users/rusi2317/projects/meta_analysis_qc/hg38/processed_data/counts/genes/'
filt_stranded_genes <- 'filt.5ptrunc_gene.stranded_counts.txt$'

##all counts 
combine_anno_counts(file_directory=shares,
                    file_pattern=filt_stranded_genes,  
                    out_directory=wd, 
                    out_file='counts_filt_5ptrunc_gene_stranded_counts.txt',      
                    ntranscripts=nrow(example_sample),  
                    transcript_beds_DT=hg38_refseq_bed,
                    sample=example_sample
                        )


[1] "Input files : 2395"
[1] "Loaded files matching # transcripts : 1645"
[1] "Did the same number of files get loaded? :  FALSE"
[1] "The difference is due to human vs. mouse samples in each folder"
[1] "Number of samples =  1645"
[1] "Number of columns =  1651"
[1] "--------------------------"
[1] "> Columns AFTER filtering"
   [1] "chrom"           "start"           "stop"            "gene_transcript"
   [5] "score"           "strand"          "SRR7266931"      "SRR7266932"     
   [9] "SRR7266933"      "SRR7266934"      "SRR7266935"      "SRR7266936"     
  [13] "SRR7266937"      "SRR7266938"      "SRR1105736"      "SRR1105737"     
  [17] "SRR1105738"      "SRR1105739"      "SRR1105740"      "SRR1105741"     
  [21] "SRR1224573"      "SRR1224574"      "SRR1596500"      "SRR1596501"     
  [25] "SRR1806546"      "SRR1806547"      "SRR1806548"      "SRR1806549"     
  [29] "SRR1806550"      "SRR1806551"      "SRR1806552"      "SRR1806553"     
  [33] "SRR1806554"      "SRR1806555"  

### Full length

In [11]:
filt_stranded_genes_full <- 'filt.gene.stranded_counts.txt$'

##all counts 
combine_anno_counts(file_directory=shares,
                    file_pattern=filt_stranded_genes_full,  
                    out_directory=wd, 
                    out_file='counts_filt_gene_stranded_counts.txt',      
                    ntranscripts=nrow(example_sample_full),  
                    transcript_beds_DT=hg38_refseq_full_bed,
                    sample=example_sample
                        )


[1] "Input files : 2395"
[1] "Loaded files matching # transcripts : 1645"
[1] "Did the same number of files get loaded? :  FALSE"
[1] "The difference is due to human vs. mouse samples in each folder"
[1] "Number of samples =  1645"
[1] "Number of columns =  1651"
[1] "--------------------------"
[1] "> Columns AFTER filtering"
   [1] "chrom"           "start"           "stop"            "gene_transcript"
   [5] "score"           "strand"          "SRR7266931"      "SRR7266932"     
   [9] "SRR7266933"      "SRR7266934"      "SRR7266935"      "SRR7266936"     
  [13] "SRR7266937"      "SRR7266938"      "SRR1105736"      "SRR1105737"     
  [17] "SRR1105738"      "SRR1105739"      "SRR1105740"      "SRR1105741"     
  [21] "SRR1224573"      "SRR1224574"      "SRR1596500"      "SRR1596501"     
  [25] "SRR1806546"      "SRR1806547"      "SRR1806548"      "SRR1806549"     
  [29] "SRR1806550"      "SRR1806551"      "SRR1806552"      "SRR1806553"     
  [33] "SRR1806554"      "SRR1806555"  

## Bidirectionals

In [12]:
shares <- '/Shares/dbnascent'
bidir_wd <- '/Users/rusi2317/projects/meta_analysis_qc/hg38/processed_data/counts/bidirectionals/'
bidir_sense <- 'bidir.pos_counts.txt$'
bidir_antisense <- 'bidir.neg_counts.txt$'

##sense 
combine_bidir_counts(file_directory=shares, 
                     file_pattern=bidir_sense, 
                     out_directory=bidir_wd, 
                     out_file='counts_bidirectionals_pos.bed', 
                     ntranscripts=nrow(hg38_bidirectionals), 
                     transcript_beds_DT=hg38_bidirectionals)

##antisense
combine_bidir_counts(file_directory=shares, 
                     file_pattern=bidir_antisense, 
                     out_directory=bidir_wd, 
                     out_file='counts_bidirectionals_neg.bed', 
                     ntranscripts=nrow(hg38_bidirectionals), 
                     transcript_beds_DT=hg38_bidirectionals)

[1] "--------------------------"
[1] "Input files : 2395"
[1] "Loaded files matching # transcripts : 1645"
[1] "Did the same number of files get loaded? :  FALSE"
[1] "The difference is due to human vs. mouse samples in each folder"
[1] "Number of samples =  1645"
[1] "--------------------------"
[1] "--------------------------"
[1] "Input files : 2395"
[1] "Loaded files matching # transcripts : 1645"
[1] "Did the same number of files get loaded? :  FALSE"
[1] "The difference is due to human vs. mouse samples in each folder"
[1] "Number of samples =  1645"
[1] "--------------------------"


# Session Information

In [14]:
sessionInfo()

R version 3.6.0 (2019-04-26)
Platform: x86_64-redhat-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /usr/lib64/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] data.table_1.14.2 tidyr_1.2.1       dplyr_1.0.10     

loaded via a namespace (and not attached):
 [1] magrittr_2.0.3   tidyselect_1.1.2 uuid_1.1-0       R6_2.5.1        
 [5] rlang_1.0.6      fastmap_1.1.0    fansi_1.0.3      tools_3.6.0     
 [9] utf8_1.2.2       DBI_1.1.3        cli_3.4.1        htmltools_0.5.2 
[13] asserttha